# Views are different perspectives of our data
That would be a way to get a better understanding of the data.  
If you have an idea for a view, share it or make an issue.

In [2]:
import altair as alt
import numpy as np
import pandas as pd

from mlrepricer.oldsql import schemas
from mlrepricer import setup, helper
from mlrepricer.oldsql.database import SQLite
alt.data_transformers.enable('default', max_rows=1000000)

DataTransformerRegistry.enable('default')

In [3]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

### Filter1 aka features we probably should not track in the first place.
for me it drops like 4% of the data

In [4]:
# this is available in mlrepricer.helper.cleanup()
filter1 = df[(df.instock==1)&(df.isfeaturedmerchant==1)]
filter1 = filter1.drop(['instock', 'isfeaturedmerchant'], axis=1)
# those offers suck too
filter1 = filter1[(filter1.shipping_maxhours+filter1.shipping_minhours)<=72]

We look at each attribute separetely now.  
Basically it's ok to drop both features.  
The instock attribute has some winners, but it's to unlikely.  
It's like 1/300 of the dataset got that feature.

In [5]:
drop2 = df[df.isfeaturedmerchant==0]  # it has a zero chance to be a winner
print(f'We can drop {len(drop2)} rows and loose {len(drop2[drop2.isbuyboxwinner==1])} winners, we expected this to be Zero')

We can drop 10429 rows and loose 0 winners, we expected this to be Zero


In [6]:

drop = df[df.instock==0]
normalized_lostwinner = len(filter1[filter1.isbuyboxwinner==1])/len(filter1)/(len(drop[drop.isbuyboxwinner==1])/len(drop))
print(f'Its {normalized_lostwinner:.2f} times less likely to find a buyboxwinner in the data set we drop here, we have {len(drop)} datarows.')
if normalized_lostwinner < 5:
    raise ValueError

Its 6.49 times less likely to find a buyboxwinner in the data set we drop here, we have 1022 datarows.


# The stuff below is not wrong, but it makes only little sense to look at the data this way.
Better just look on a per message level. Because one message has all the information for a state.

## What is the name for the type of feed data we get?
It's like time series data, but tracking only changes.
I think without reducing the complexity we can imagine it in 5 dimensions.
 - asin
 - time_changed  
   competitor = groupby(['sellerid', 'isprime']).min()  # min because maybe some have duplicates offers  
     those are two seperate dimensions:
     - sellerid
     - isprime
 - features
 
does this make sense?

Knowing this we can make a pivot table. Let's refer to it as groundtruth.

In [8]:
filter1.groupby(['asin', 'sellerid', 'isprime', 'time_changed']).min()

feedback  \
asin       sellerid       isprime time_changed                        
B0126QIK2K A2EHWGAW6J9W8Q 0       2018-05-22 07:33:12.314      1606   
                                  2018-05-22 07:42:09.934      1606   
                                  2018-05-22 09:38:58.802      1606   
           AFEE1JGTCYXOG  0       2018-05-22 07:33:12.314      2072   
                                  2018-05-22 07:42:09.934      2072   
                                  2018-05-22 09:38:58.802      2072   
B015NJUI5O A10GXGUDASI5WW 1       2018-05-10 02:03:16.011       503   
                                  2018-05-10 06:03:58.780       503   
                                  2018-05-10 13:23:17.559       503   
                                  2018-05-10 20:30:04.405       504   
                                  2018-05-10 20:32:14.141       504   
                                  2018-05-11 02:03:08.665       504   
                                  2018-05-11 02:21:48.791       504   
                                  2018-05-11 03:33:24.403       504   
                                  2018-05-11 03:33:25.253       504   
                                  2018-05-11 13:23:20.810       506   
                                  2018-05-11 13:26:11.090       506   
                                  2018-05-11 20:29:20.611       507   
                                  2018-05-11 20:33:10.628       507   
                                  2018-05-12 03:30:46.468       506   
                                  2018-05-12 03:34:48.415       506   
                                  2018-05-12 20:28:54.822       506   
                                  2018-05-12 20:34:14.535       506   
                                  2018-05-13 02:01:55.684       507   
                                  2018-05-13 02:03:10.495       507   
                                  2018-05-13 03:32:29.917       507   
                                  2018-05-13 03:34:15.219       507   
                                  2018-05-13 13:25:10.797       507   
                                  2018-05-13 13:37:01.215       507   
                                  2018-05-13 20:30:40.607       507   
...                                                             ...   
B07B11MPC4 ADBIN7JOKZABU  0       2018-06-05 16:54:48.097      6687   
                                  2018-06-05 17:08:24.017      6687   
                                  2018-06-09 08:40:31.669      6720   
                                  2018-06-09 09:44:22.818      6721   
                                  2018-06-16 06:33:32.445      6774   
                                  2018-06-16 06:44:38.546      6774   
                                  2018-06-18 18:52:25.425      6793   
                                  2018-06-18 19:54:34.724      6793   
B07B8NGM2J A16ILHN79BWR3Q 0       2018-06-18 14:54:16.913       112   
           A2EHWGAW6J9W8Q 0       2018-06-18 14:54:16.913      1618   
B07B9Q4RS1 A2EHWGAW6J9W8Q 0       2018-06-16 01:18:08.154      1616   
                                  2018-06-16 01:18:21.532      1616   
                                  2018-06-17 16:58:57.638      1618   
           A2T7HYLM0MW6DH 1       2018-06-16 01:18:21.532        85   
                                  2018-06-17 16:58:57.638        85   
B07C5FHXKM A2EHWGAW6J9W8Q 0       2018-06-18 02:03:28.303      1618   
                                  2018-06-18 02:03:43.783      1618   
                                  2018-06-18 19:10:36.766      1618   
                                  2018-06-18 20:52:01.548      1618   
                          1       2018-06-18 02:03:28.303      1618   
                                  2018-06-18 02:03:43.783      1618   
                                  2018-06-18 19:10:36.766      1618   
                                  2018-06-18 20:52:01.548      1618   
           AJXY5LDG2OJ56  1       2018-06-18 02:03:28.303         3   
            

What can we improve from here?
 - only track stable states, like those hold for more than 5 minutes
 - linear transform isprime to remove a dimension  
More ideas are welcome on github.